# XOR

Can we make `xor` with a net?

In [43]:
%matplotlib inline

In [44]:
import numpy as np
import matplotlib.pyplot as plt

`xor(x,y)` is x != y, viz:

In [45]:
True ^ True, False ^ False, True ^ False, False ^ True

(False, False, True, True)

Use -1 to represent one boolean condition, and +1 the other. Then

In [46]:
def flexor(a:float, b:float) -> float:
    return a*b < 0

In [47]:
flexor(-1, -1), flexor(1,1), flexor(-1,1), flexor(1,-1)

(False, False, True, True)

## Support functions
We need some functions implementing nonlinear operations:

In [48]:
def relu(x):
    #return (lambda v: max(0,v))(x)
    return np.vectorize(lambda v: max(0.0,v))(x)

[(v, relu(v)) for v in np.arange(-2, 2, 0.5)]

[(-2.0, array(0.)),
 (-1.5, array(0.)),
 (-1.0, array(0.)),
 (-0.5, array(0.)),
 (0.0, array(0.)),
 (0.5, array(0.5)),
 (1.0, array(1.)),
 (1.5, array(1.5))]

In [49]:
def positive(x):
    return np.vectorize(lambda v: max(0, np.sign(v)))(x)

[(v, positive(v)) for v in np.arange(-2, 2, 0.5)]

[(-2.0, array(0)),
 (-1.5, array(0)),
 (-1.0, array(0)),
 (-0.5, array(0)),
 (0.0, array(0)),
 (0.5, array(1.)),
 (1.0, array(1.)),
 (1.5, array(1.))]

In [50]:
def tanh_gradient(x):
    return 1-np.tanh(x)**2

In [51]:
def frombits(v) -> int:
    p = 1
    s = 0
    for bit in v:
        s += p * bit
        p <<= 1
    return s

In [52]:
frombits([1,0,1,1])

13

## Network implementations

In [ ]:
class Exactor:
    """Calculate exclusive-or using a network"""
    def __init__(self):
        self.randomize()
        
    def randomize(self):
        "Randomize the matricies"
        self.m1 = np.random.randn(2,2)
        self.m2 = np.random.randn(2)
    
    def make_perfect(self):
        "Set the matricies to a handmade value that gives perfect behavior"
        self.m1 = np.array([[ 1.0, -1.0],
                            [-1.0,  1.0]])
        self.m2 = np.array([1.0, 1.0])
    
    def ideal(self, a:float, b:float) -> bool:
        "Calculates the ideal return value directly, to provide a reference"
        return (-1.0,1.0)[a*b < 0]
        #return np.sign(-a*b)
    
    def netwise(self, a:float, b:float):
        "Calculate a single result using network primitives"
        v = self.net_lin(a, b)
        v = self.p5 = (-1,1)[int(np.sign(v-0.5)[0])]
        self.p5 = v
        return v

    def net_ana(self, a:float, b:float):
        "Calculate a single analog result using network primitives"
        v = self.p4 = np.tanh(self.net_lin(a, b))[0]
        return v

    def net_lin(self, a:float, b:float):
        "The network output up to the last linear stage"
        input = np.array([[a],
                          [b]])
        v = self.p1 = self.m1 @ input
        v = self.p2 = relu(v)
        v = self.p3 = np.dot(self.m2, v)
        return v

    def __call__(self, a, b, analog=False):
        "Vectorized calculation of result using network"
        return np.vectorize(self.netwise)(a, b)

    def loss(self, analog=False):
        "L2 loss function of the network implementation"
        return sum((self.__call__(x,y, analog) - self.ideal(x,y))**2 for x in (-1, 1) for y in (-1, 1))
    
    def __repr__(self):
        return F"Exactor m1={self.m1}, m2={self.m2})"
            

### Try out the implementation

In [ ]:
exor = Exactor()
exor
exor.net_lin(1,1)

In [ ]:
exor.net_ana(1,1)

In [ ]:
exor.netwise(1,1)

In [ ]:
for x in (-1, 1):
    for y in (-1, 1):
        print(exor.ideal(x, y), exor(x,y), exor.net_ana(x,y))

In [ ]:
exor.loss(), exor.loss(analog=True)

In [ ]:
exor.make_perfect()
print(exor)

In [ ]:
for x in (-1, 1):
    for y in (-1, 1):
        print(exor.ideal(x, y), exor(x,y))

In [ ]:
exor.loss(), exor.loss(analog=True)

In [ ]:
exor.randomize()
exor.loss(), exor.loss(analog=True)

### Can we find working matricies by trying random matricies?

In [ ]:
best_loss = 1e300
best_repr = ""
for n in range(10000):
    exor.randomize()
    #if n == 6789:
    #    exor.make_perfect()
    loss = exor.loss()
    if best_loss > loss:
        best_loss = loss
        best_repr = repr(exor)
    if loss == 0:
        print(F"Success after {n+1} tries: {exor}")
        break
if exor.loss() > 0:
    print(F"Failure, none of {n+1} random tries worked")
print(f"best net loss:{best_loss}, Best net:{best_repr}")

### A less delicate network implementation

In [ ]:
class Flexor:
    """A more robust XOR
    Returns the sign of (|x-y| - |x+y|) where x, y are inputs"""
    def __init__(self):
        self.randomize()
        
    def randomize(self):
        "Randomize the matricies"
        self.m1 = np.random.randn(4,2)
        self.m2 = np.random.randn(4)
    
    def make_perfect(self):
        "Set the matricies to a handmade value that gives perfect behavior"
        self.m1 = np.array([[ 1.0, -1.0],
                            [-1.0,  1.0],
                            [ 1.0,  1.0],
                            [-1.0, -1.0]])
        self.m2 = np.array([1.0, 1.0, -1.0, -1.0])
    
    def ideal(self, a:float, b:float) -> bool:
        "Calculates the ideal return value directly, to provide a reference"
        return np.sign(-a*b)
    
    def netwise(self, a:float, b:float) -> bool:
        "Calculate a single result using network primitives"
        input = np.array([[a],
                          [b]])
        v = self.p1 = self.m1 @ input
        v = self.p2 = relu(v)
        v = self.p3 = np.dot(self.m2, v)
        v = self.p4 = relu(v)
        v = self.p5 = (-1,1)[int(np.sign(v)[0])]
        return v

    def __call__(self, a, b):
        "Vectorized calculation of result using network"
        return np.vectorize(self.netwise)(a, b)

    def loss(self):
        "L2 loss function of the network implementation"
        return sum((self.__call__(x,y) - self.ideal(x,y))**2 for x in (-1, 1) for y in (-1, 1))
    
    def goodness(self):
        "analog goodness function"
        rv = 0.0
        for x in (-1, 1):
            for y in (-1, 1):
                _ = self.netwise(x, y)
                rv += self.p4 * self.ideal(x,y)
        return rv[0]

    def __repr__(self):
        return F"Flexor(m1={self.m1}, m2={self.m2})"

In [ ]:
flor = Flexor()
flor.m1, flor.m2

In [ ]:
for x in (-1, 1):
    for y in (-1, 1):
        print(flor.ideal(x, y), '\tres:', flor(x,y))

In [ ]:
flor.loss(), flor.goodness()

In [ ]:
flor.make_perfect()

In [ ]:
for x in (-1, 1):
    for y in (-1, 1):
        print(flor.ideal(x, y), flor(x,y))

In [ ]:
flor.loss(), flor.goodness()

In [ ]:
flor.randomize()
flor.loss(), flor.goodness()

### Can we find working matricies by trying random matricies?

In [ ]:
for n in range(10000):
    flor.randomize()
    if flor.loss() == 0:
        print(F"Success after {n+1} tries:\n{flor}")
        break
if flor.loss() > 0:
    print(F"Failure, none of {n+1} random tries worked")

In [ ]:
flor.loss(), flor.goodness()